оценка от 0 до 5

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Пример данных: матрица взаимодействий пользователей и объектов
# 1 - пользователь взаимодействовал с объектом, 0 - нет
interactions = np.array([
    [1, 1, 0, 0],  # Пользователь 0
    [1, 0, 0, 1],  # Пользователь 1
    [0, 1, 1, 0],  # Пользователь 2
    [0, 0, 1, 1],  # Пользователь 3
], dtype=np.float32)

# Гиперпараметры
n_users, n_items = interactions.shape
latent_dim = 10  # Количество латентных факторов
learning_rate = 0.01
n_epochs = 100

# Модель BPR
class BPR(nn.Module):
    def __init__(self, n_users, n_items, latent_dim):
        super(BPR, self).__init__()
        self.user_factors = nn.Embedding(n_users, latent_dim)
        self.item_factors = nn.Embedding(n_items, latent_dim)
        # Инициализация весов
        nn.init.normal_(self.user_factors.weight, mean=0, std=0.01)
        nn.init.normal_(self.item_factors.weight, mean=0, std=0.01)

    def forward(self, u, i, j):
        # Получаем латентные факторы для пользователей и объектов
        u_factors = self.user_factors(u)
        i_factors = self.item_factors(i)
        j_factors = self.item_factors(j)
        # Вычисляем предсказания
        x_ui = torch.sum(u_factors * i_factors, dim=1)
        x_uj = torch.sum(u_factors * j_factors, dim=1)
        return x_ui - x_uj

# Функция потерь BPR
def bpr_loss(pred):
    return -torch.log(torch.sigmoid(pred)).mean()

# Подготовка данных
def generate_triples(interactions):
    triples = []
    for u in range(n_users):
        pos_items = np.where(interactions[u] == 1)[0]
        neg_items = np.where(interactions[u] == 0)[0]
        for i in pos_items:
            for j in neg_items:
                triples.append((u, i, j))
    return np.array(triples)

# Генерация троек (u, i, j)
triples = generate_triples(interactions)

# Преобразуем данные в тензоры
triples = torch.tensor(triples, dtype=torch.long)

# Модель и оптимизатор
model = BPR(n_users, n_items, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    # Перемешиваем тройки
    idx = torch.randperm(len(triples))
    for batch in torch.split(triples[idx], 64):  # Мини-батчи
        u, i, j = batch[:, 0], batch[:, 1], batch[:, 2]
        pred = model(u, i, j)
        loss = bpr_loss(pred)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Предсказание оценок
def predict_ratings(model, user_id):
    model.eval()
    with torch.no_grad():
        u = torch.tensor([user_id], dtype=torch.long)
        i = torch.arange(n_items, dtype=torch.long)
        scores = torch.sum(model.user_factors(u) * model.item_factors(i), dim=1)
    return scores.numpy()

# Нормализация оценок
def normalize_scores(scores, min_val=0, max_val=5):
    scores_min = scores.min()
    scores_max = scores.max()
    return min_val + (scores - scores_min) * (max_val - min_val) / (scores_max - scores_min)

# Пример предсказания для пользователя 0
user_id = 0
predicted_scores = predict_ratings(model, user_id)
normalized_scores = normalize_scores(predicted_scores)
print(f"Предсказанные оценки для пользователя {user_id}: {predicted_scores}")
print(f"Нормализованные оценки для пользователя {user_id}: {normalized_scores}")

Epoch 1, Loss: 0.693071186542511
Epoch 2, Loss: 0.692124605178833
Epoch 3, Loss: 0.6904391050338745
Epoch 4, Loss: 0.6878796219825745
Epoch 5, Loss: 0.6844171285629272
Epoch 6, Loss: 0.6800409555435181
Epoch 7, Loss: 0.6747444868087769
Epoch 8, Loss: 0.6685162782669067
Epoch 9, Loss: 0.6613413691520691
Epoch 10, Loss: 0.653204619884491
Epoch 11, Loss: 0.6440922617912292
Epoch 12, Loss: 0.6339951753616333
Epoch 13, Loss: 0.6229115724563599
Epoch 14, Loss: 0.6108473539352417
Epoch 15, Loss: 0.5978164076805115
Epoch 16, Loss: 0.5838404893875122
Epoch 17, Loss: 0.5689497590065002
Epoch 18, Loss: 0.5531820058822632
Epoch 19, Loss: 0.5365827083587646
Epoch 20, Loss: 0.5192041993141174
Epoch 21, Loss: 0.5011062622070312
Epoch 22, Loss: 0.4823554456233978
Epoch 23, Loss: 0.46302616596221924
Epoch 24, Loss: 0.4432011842727661
Epoch 25, Loss: 0.42297205328941345
Epoch 26, Loss: 0.40243929624557495
Epoch 27, Loss: 0.38171207904815674
Epoch 28, Loss: 0.3609068989753723
Epoch 29, Loss: 0.3401457369